# Dados Disponiveis em https://data.insideairbnb.com/italy/toscana/florence/2025-06-19/data/listings.csv.gz
## Florence,Toscana, Italy

In [ ]:
# === 01 • Imports e paths ===
import os
import re
import numpy as np
import pandas as pd

# Caminhos do projeto
DATA_RAW = "../data/raw/listings.csv"      # ajuste se seu notebook estiver em outra pasta
DATA_OUT_DIR = "../data/processed"
os.makedirs(DATA_OUT_DIR, exist_ok=True)

pd.set_option("display.max_columns", 100)
pd.set_option("display.float_format", lambda x: f"{x:,.2f}")


In [ ]:
# === 02 • Carga do CSV e inspeção rápida ===
df = pd.read_csv(DATA_RAW, low_memory=False)
print(f"Shape bruto: {df.shape[0]} linhas × {df.shape[1]} colunas")

# Amostra e colunas
display(pd.DataFrame({"coluna": df.columns}))
display(df.dtypes)
display(df.head(3))



In [ ]:
df.describe().T  #Estatísticas básicas (para colunas numéricas)

In [ ]:
# === 03 • Helpers de limpeza ===

def to_float_price(series):
    """Converte preço textual ('$1,234.00') para float. Mantém NaN se vazio."""
    return (
        series.astype(str)
              .str.replace(r"[^\d\.\,]", "", regex=True)   # remove tudo que não é número/ponto/vírgula
              .str.replace(",", "", regex=False)           # remove separador de milhar
              .replace({"": np.nan, "nan": np.nan})
              .astype(float)
    )

def coerce_numeric(series):
    """Converte para numérico de forma tolerante (erros viram NaN)."""
    return pd.to_numeric(series, errors="coerce")

def true_false_to_int(series):
    """Converte 't'/'f' ou True/False em 1/0."""
    return series.map({True:1, False:0, "t":1, "f":0}).astype("float")


In [ ]:
# === 04 • Limpeza de campos essenciais ===

df["price"] = to_float_price(df["price"])

# Numéricos relevantes
for col in ["accommodates","bedrooms","beds","minimum_nights","maximum_nights",
            "number_of_reviews","reviews_per_month","review_scores_rating",
            "bathrooms","bathrooms_text","availability_365","availability_90",
            "availability_30"]:
    if col in df.columns:
        df[col] = coerce_numeric(df[col])

# Bools/categóricos simples
for col in ["instant_bookable","host_is_superhost"]:
    if col in df.columns:
        df[col] = true_false_to_int(df[col])

# Latitude/Longitude
if "latitude" in df.columns: df["latitude"] = coerce_numeric(df["latitude"])
if "longitude" in df.columns: df["longitude"] = coerce_numeric(df["longitude"])

# Padroniza alguns categóricos para category (one-hot será depois)
cat_cols = ["room_type","property_type","neighbourhood_cleansed"]
for c in cat_cols:
    if c in df.columns:
        df[c] = df[c].astype("category")

print("Campos essenciais limpos.")
df[["price","accommodates","bedrooms","bathrooms","beds","room_type","property_type",
    "latitude","longitude","number_of_reviews","reviews_per_month","review_scores_rating",
    "instant_bookable","host_is_superhost","availability_365"]].head(3)


In [ ]:
# === 05 • Seleção de features para o baseline de regressão ===

FEATURES = [
    # localização
    "latitude","longitude",
    # tipo/uso
    "room_type","property_type",
    # capacidade
    "accommodates","bedrooms","bathrooms","beds",
    # reputação/demanda
    "number_of_reviews","reviews_per_month","review_scores_rating",
    # operação
    "minimum_nights","maximum_nights","availability_365",
    # conveniência/boas práticas
    "instant_bookable","host_is_superhost"
]

TARGET = "price"

# Garante que só usamos colunas que existem no dataset atual
FEATURES = [c for c in FEATURES if c in df.columns]
cols_modelo = [c for c in FEATURES] + [TARGET]

df_model = df[cols_modelo].copy()

# Remover duplicatas (caso existam)
df_model = df_model.drop_duplicates()

# Remover linhas sem target ou com lat/long ausentes (essenciais pro heatmap futuro)
req_cols = [TARGET, "latitude", "longitude"]
req_cols_existentes = [c for c in req_cols if c in df_model.columns]
df_model = df_model.dropna(subset=req_cols_existentes)

print(f"Shape df_model (pré-tratamento de NaNs em features): {df_model.shape}")
missing = df_model.isna().sum().sort_values(ascending=False)
display(missing[missing>0].to_frame("n_missing"))


In [ ]:
# === 06 • Tratamento de faltantes + cap de outliers simples ===

# 1) Drop de faltantes nas features numéricas essenciais do baseline
numeric_essentials = ["accommodates","bedrooms","bathrooms","beds",
                      "number_of_reviews","reviews_per_month",
                      "review_scores_rating","minimum_nights","maximum_nights",
                      "availability_365"]
numeric_essentials = [c for c in numeric_essentials if c in df_model.columns]

df_model = df_model.dropna(subset=[c for c in numeric_essentials if c in df_model.columns])

# 2) Cap (winsorization leve) em colunas com caudas longas
def cap_outliers(s, low_q=0.01, high_q=0.99):
    lo, hi = s.quantile([low_q, high_q])
    return s.clip(lo, hi)

to_cap = ["price","accommodates","bedrooms","bathrooms","beds",
          "minimum_nights","maximum_nights","number_of_reviews",
          "reviews_per_month","availability_365","review_scores_rating"]
to_cap = [c for c in to_cap if c in df_model.columns]

for c in to_cap:
    df_model[c] = cap_outliers(df_model[c])

# 3) Remoção de valores obviamente inválidos (exemplos)
if "price" in df_model.columns:
    df_model = df_model[df_model["price"] > 0]

if "accommodates" in df_model.columns:
    df_model = df_model[df_model["accommodates"] >= 1]

print(f"Shape df_model (pós-limpeza baseline): {df_model.shape}")
display(df_model.describe(include="all").transpose().head(20))


In [ ]:
# === 07 • Persistência para os próximos passos ===

OUT_CSV = os.path.join(DATA_OUT_DIR, "listings_model_baseline.csv")
df_model.to_csv(OUT_CSV, index=False, encoding="utf-8")
print(f"Arquivo salvo em: {OUT_CSV}")

